In [18]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

train_set = torchvision.datasets.CIFAR10('./data', train=True,
                                       download=True,
                                       transform=transforms)
train_loader = DataLoader(train_set, batch_size=4, shuffle=False)

# treat the test dataset as a validation set for this example
validation_set = torchvision.datasets.CIFAR10('./data', train=False,
                                            download=True,
                                            transform=transforms)
validation_loader = DataLoader(validation_set, batch_size=4, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training and Validation

In [ ]:
import torch


def train(net, train_loader, validation_loader, validation_during_training, num_epochs=2):
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        net.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[Epoch %d, Iter %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
            
        if validation_during_training:
            validation(net, validation_loader, epoch)
    
    print('Finished Training')



def validation(net, validation_dataloader, epoch):
    net.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validation_dataloader, 0):
            inputs, labels = data
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    print('Val [Epoch %d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / len(validation_dataloader) ))

    
print('======== Training Without Validation ========')
torch.manual_seed(3)
train(net, train_loader, validation_loader, validation_during_training=True)
print('======== Training With Validation ========')
torch.manual_seed(3)
train(net, train_loader, validation_loader, validation_during_training=False)

======== Training Without Validation ========
[Epoch 1, Iter  2000] loss: 1.191
[Epoch 1, Iter  4000] loss: 1.200
[Epoch 1, Iter  6000] loss: 1.158
[Epoch 1, Iter  8000] loss: 1.148
[Epoch 1, Iter 10000] loss: 1.189
[Epoch 1, Iter 12000] loss: 1.178
Val [Epoch 1,  2500] loss: 1.181
[Epoch 2, Iter  2000] loss: 1.153
[Epoch 2, Iter  4000] loss: 1.163
[Epoch 2, Iter  6000] loss: 1.126
[Epoch 2, Iter  8000] loss: 1.118
[Epoch 2, Iter 10000] loss: 1.157
[Epoch 2, Iter 12000] loss: 1.137
Val [Epoch 2,  2500] loss: 1.130
Finished Training
======== Training With Validation ========
[Epoch 1, Iter  2000] loss: 1.075
[Epoch 1, Iter  4000] loss: 1.077
[Epoch 1, Iter  6000] loss: 1.047
[Epoch 1, Iter  8000] loss: 1.036
